<a href="https://colab.research.google.com/github/daakanksha/Fruits-classification-by-transfer-learning/blob/main/image_classification_cnn_%26_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = '/content/drive/MyDrive/verzeo/Artificial intelligence/major'

In [ ]:
#import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K

In [ ]:
# Loading data and putting them into training and test sets
#locations setting for training and test datasets
train_data='/content/drive/MyDrive/verzeo/Artificial intelligence/major/fruits-360/Training'
test_data='/content/drive/MyDrive/verzeo/Artificial intelligence/major/fruits-360/Test'
#creates X_train and Y_train using file_names and folders
def get_data(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files,targets,target_labels
X_train, Y_train, labels = get_data(train_data)
X_test, Y_test,_ = get_data(test_data)

In [ ]:
Y_train = np_utils.to_categorical(Y_train, 120)
Y_test = np_utils.to_categorical(Y_test, 120)

In [ ]:
# splitting train set into training and validation sets
X_train, X_val = train_test_split(X_train, test_size=0.2, random_state=33)
Y_train, Y_val = train_test_split(Y_train, test_size=0.2, random_state=33)

In [ ]:
#converting images into array to start computation
def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array
X_train = np.array(convert_image_to_array(X_train))
X_val = np.array(convert_image_to_array(X_val))
X_test = np.array(convert_image_to_array(X_test))

In [ ]:
#nomalizing the pixel values before feeding into a neural network
X_train = X_train.astype('float32')/255
X_val = X_val.astype('float32')/255
X_test = X_test.astype('float32')/255

In [ ]:
#Building model 1 using customized convolutional and pooling layers
model = Sequential()
#input_shape is 100*100 since thats the dimension of each of the fruit images
model.add(Conv2D(filters = 16, kernel_size = 2,input_shape=(100,100,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters = 32,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters = 64,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters = 128,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))
# specifying parameters for fully connected layer
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(120,activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 16)      208       
_________________________________________________________________
activation (Activation)      (None, 100, 100, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0

In [ ]:
#importing ootimizers
from keras.optimizers import SGD, Adam, RMSprop
optimizer = Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
# creating a file to save the trained CNN model 
checkpointer = ModelCheckpoint(filepath = 'cnn_from_scratch_fruits.hdf5', verbose = 1, save_best_only = True)
# fitting model using above defined layers 
CNN_model = model.fit(X_train,Y_train,
        batch_size = 128,
        epochs=20,
        validation_data=(X_val, Y_val),
        callbacks = [checkpointer],
        verbose=2, shuffle=True)

Epoch 1/20
15/15 - 48s - loss: 2.8441 - accuracy: 0.2219 - val_loss: 1.5020 - val_accuracy: 0.4586

Epoch 00001: val_loss improved from inf to 1.50202, saving model to cnn_from_scratch_fruits.hdf5
Epoch 2/20
15/15 - 14s - loss: 1.4806 - accuracy: 0.4437 - val_loss: 0.8006 - val_accuracy: 0.8726

Epoch 00002: val_loss improved from 1.50202 to 0.80057, saving model to cnn_from_scratch_fruits.hdf5
Epoch 3/20
15/15 - 14s - loss: 0.7472 - accuracy: 0.7022 - val_loss: 0.3031 - val_accuracy: 0.9936

Epoch 00003: val_loss improved from 0.80057 to 0.30309, saving model to cnn_from_scratch_fruits.hdf5
Epoch 4/20
15/15 - 14s - loss: 0.2684 - accuracy: 0.9437 - val_loss: 0.0605 - val_accuracy: 0.9936

Epoch 00004: val_loss improved from 0.30309 to 0.06054, saving model to cnn_from_scratch_fruits.hdf5
Epoch 5/20
15/15 - 15s - loss: 0.1054 - accuracy: 0.9788 - val_loss: 0.0300 - val_accuracy: 0.9936

Epoch 00005: val_loss improved from 0.06054 to 0.02999, saving model to cnn_from_scratch_fruits.hdf5

In [ ]:
#loading saved weights to use further
model.load_weights('cnn_from_scratch_fruits.hdf5')

In [ ]:
#importing vgg16
#Part 2 using transfer learning
#importing vgg16 architecture which is trained on Imagenet
from keras.applications.vgg16 import VGG16
vgg_model = VGG16(input_shape=[100,100,3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg_model.layers:
   layer.trainable = False
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [ ]:
transfer_learning_model = Sequential()
transfer_learning_model.add(vgg_model)
transfer_learning_model.add(Conv2D(1024, kernel_size=3, padding='same'))
transfer_learning_model.add(Activation('relu'))
transfer_learning_model.add(MaxPooling2D(pool_size=(2, 2)))
transfer_learning_model.add(Dropout(0.3))
transfer_learning_model.add(Flatten())
transfer_learning_model.add(Dense(150))
transfer_learning_model.add(Activation('relu'))
transfer_learning_model.add(Dropout(0.4))
transfer_learning_model.add(Dense(120,activation = 'softmax'))
transfer_learning_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 3, 3, 512)         14714688  
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 1024)        4719616   
_________________________________________________________________
activation_2 (Activation)    (None, 3, 3, 1024)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 1, 1024)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 1, 1024)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 150)              

In [ ]:
from keras.optimizers import SGD, Adam, RMSprop
optimizer = Adam()
transfer_learning_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
#fitting the new model
checkpointer = ModelCheckpoint(filepath = 'transfer_learning.hdf5', verbose = 1, save_best_only = True)
# running 
transfer_learning_cnn = transfer_learning_model.fit(X_train,Y_train,
        batch_size = 128,
        epochs=20,
        validation_data=(X_val, Y_val),
        callbacks = [checkpointer],
        verbose=2, shuffle=True)

Epoch 1/20
15/15 - 247s - loss: 1.7372 - accuracy: 0.5069 - val_loss: 0.3457 - val_accuracy: 0.8514

Epoch 00001: val_loss improved from inf to 0.34566, saving model to transfer_learning.hdf5
Epoch 2/20
15/15 - 241s - loss: 0.3385 - accuracy: 0.8944 - val_loss: 0.0474 - val_accuracy: 0.9915

Epoch 00002: val_loss improved from 0.34566 to 0.04737, saving model to transfer_learning.hdf5
Epoch 3/20
15/15 - 238s - loss: 0.1023 - accuracy: 0.9724 - val_loss: 0.0215 - val_accuracy: 0.9958

Epoch 00003: val_loss improved from 0.04737 to 0.02151, saving model to transfer_learning.hdf5
Epoch 4/20
15/15 - 238s - loss: 0.0521 - accuracy: 0.9873 - val_loss: 0.0054 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.02151 to 0.00544, saving model to transfer_learning.hdf5
Epoch 5/20
15/15 - 242s - loss: 0.0179 - accuracy: 0.9973 - val_loss: 0.0016 - val_accuracy: 1.0000

Epoch 00005: val_loss improved from 0.00544 to 0.00164, saving model to transfer_learning.hdf5
Epoch 6/20
15/15 - 241s 

In [ ]:
transfer_learning_model.load_weights('transfer_learning.hdf5')